In [82]:
from nilearn.glm.first_level import first_level_from_bids
from nilearn.glm.second_level import SecondLevelModel
import numpy as np
import pandas as pd
from pathlib import Path

In [83]:
from bids import BIDSLayout

In [84]:
from tqdm import tqdm

In [85]:
import nibabel as nib

In [86]:
layout =  BIDSLayout("tutorial_data/ccsl_prl",derivatives=True)

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


In [87]:
isinstance(Path('.'),Path)

True

In [88]:
task_name = 'prl'
space_name = 'MNI152NLin2009cAsym'
smoothing_fwhm = 6.0
confounds = ['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']
process_name = 'PEchosen'

In [89]:
mbmvpa_layout = layout.derivatives['MB-MVPA']

In [90]:
models, models_run_imgs, \
            models_events, models_confounds = first_level_from_bids(layout.root,
                                                                    task_name,
                                                                    space_name,
                                                                    smoothing_fwhm=smoothing_fwhm,
                                                                    derivatives_folder=layout.derivatives['fMRIPrep'].root)

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/first_level/first_level.py:853: UserWarning: SliceTimingRef not found in file /home/cheoljun/project_model_based_fmri/tutorials/tutorial_data/ccsl_prl/derivatives/fmriprep/sub-01/ses-01/func/sub-01_ses-01_task-prl_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.json. It will be assumed that the slice timing reference is 0.0 percent of the repetition time. If it is not the case it will need to be set manually in the generated list of models
  img_specs[0])


In [91]:
for i in range(len(models_confounds)):
    for j in range(len(models_confounds[i])):
        mc = models_confounds[i][j]
        mc = mc[confounds]
        models_confounds[i][j] = mc

In [92]:
def get_entity(img_path):
    filename = Path(img_path).stem
    entity = {}
    for z in filename.split('_'):
        if '-' in z:
            key,val = z.split('-')
            entity[key] = val
    return entity

In [93]:
for i in range(len(models_run_imgs)):
    
    for j in range(len(models_run_imgs[i])):
        entity = get_entity(models_run_imgs[i][j])
        kwargs = {}
        if 'ses' in entity.keys():
            kwargs['session'] = entity['ses']
        if 'run' in entity.keys():
            kwargs['run'] = entity['run']
        kwargs['subject'] = entity['sub']

        kwargs['task'] = task_name
        kwargs['desc'] = process_name
        kwargs['suffix'] = 'modulation'
        md = mbmvpa_layout.get(**kwargs)[0]
        md = pd.read_table(md)
        md['trial_type'] = [process_name]*len(md)
        models_events[i][j] = md

In [94]:
first_level_models = [models[i].fit([nib.load(run_img) for run_img in models_run_imgs[i]], events=models_events[i], confounds=models_confounds[i]) \
                          for i in tqdm(range(len(models)))]

  0%|          | 0/2 [00:00<?, ?it/s]/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:63: UserWarning: 'modulation' column found in the given events data.
  warnings.warn("'modulation' column found in the given events data.")
/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:63: UserWarning: 'modulation' column found in the given events data.
  warnings.warn("'modulation' column found in the given events data.")
 50%|█████     | 1/2 [02:16<02:16, 136.93s/it]/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:63: UserWarning: 'modulation' column found in the given events data.
  warnings.warn("'modulation' column found in the given events data.")
/home/cheoljun/.cache/pypoetry/virtualenv

In [19]:
first_level_models[0]

FirstLevelModel(signal_scaling=True, smoothing_fwhm=6.0, subject_label='01',
                t_r=1.5)

In [23]:
from nilearn.reporting import plot_design_matrix

ImportError: cannot import name 'plot_design_matrix' from 'nilearn.reporting' (/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/reporting/__init__.py)

In [28]:
import matplotlib.pyplot as plt

In [41]:
first_level_model = first_level_models[0]

In [53]:
len(first_level_model.design_matrices_[i].columns)

17

In [74]:
contrast_def = [np.zeros( len(dm.columns)) for dm in first_level_model.design_matrices_]

In [79]:
for i, dm in enumerate(first_level_model.design_matrices_):
    contrast_def[i][dm.columns.get_loc(process_name)] = 1

0

In [56]:
contrast_def  = [1] + [0] * (n_col-1)

In [58]:
contrast_def = [ np.array([1] + [0]*(17-1)),np.array([1] + [0]*(17-1))]

In [73]:
t_map = first_level_model.compute_contrast(contrast_def=contrast_def, output_type='z_score')

In [35]:
first_level_models[0]

FirstLevelModel(signal_scaling=True, smoothing_fwhm=6.0, subject_label='01',
                t_r=1.5)

In [80]:
first_level_models[0].subject_label

'01'

In [21]:
dir(first_level_models[0])


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_check_n_features',
 '_get_param_names',
 '_get_tags',
 '_get_voxelwise_model_attribute',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_data',
 'compute_contrast',
 'design_matrices_',
 'drift_model',
 'drift_order',
 'fir_delays',
 'fit',
 'fit_transform',
 'generate_report',
 'get_params',
 'high_pass',
 'hrf_model',
 'labels_',
 'mask_img',
 'masker_',
 'memory',
 'memory_level',
 'min_onset',
 'minimize_memory',
 'n_jobs',
 'noise_model',
 'predicted',
 'r_square',
 'residuals',
 'results_',
 'scaling_axis',
 'set_params',
 'signal_s

In [ ]:
def get_entity(img_path):
    filename = Path(img_path).stem
    entity = {}
    for z in filename.split('_'):
        if '-' in z:
            key,val = z.split('-')
            entity[key] = val
    return entity

In [ ]:
entity = get_entity(models_run_imgs[0][0])

In [ ]:
entity

In [ ]:
kwargs = {}
if 'ses' in entity.keys():
    kwargs['session'] = entity['ses']
if 'run' in entity.keys():
    kwargs['run'] = entity['run']
kwargs['subject'] = entity['sub']

kwargs['task'] = task_name
kwargs['desc'] = process_name
kwargs['suffix'] = 'modulation'

In [ ]:
pd.read_table(mbmvpa_layout.get(**kwargs)[0])